# Cognitive Services Personalizer 
https://github.com/Azure-Samples/cognitive-services-personalizer-samples

In this example, we will use Azure Personalizer Service to predict what Coffee a person (Alice, Bob, Cathy and Dave) prefers given the weather condition and time of day. File "example.json" contains their preferred choices of Coffee (set deterministically for the simplicity of this example). We will compare this data with the predictions from the service and generate rewards (0 or 1) based on the match and send it back to the service for training the model.

Note that a model is exported every 5 minutes (current default) if you are using the Cognitive Services instance of the Personalizer service, so you need to wait at least until that time has expired then to actually observe some learning in the rewards returned. Exploration is set at 20%.

Current implementation calls the Personalizer service through http calls. We will replace this with a python client in the future.

In [0]:
!pip install azure-cognitiveservices-personalizer

In [0]:
from azure.cognitiveservices.personalizer import PersonalizerClient
from azure.cognitiveservices.personalizer.models import RankableAction, RewardRequest, RankRequest
from msrest.authentication import CognitiveServicesCredentials

import json
import matplotlib.pyplot as plt
import random 
import time
import uuid

In [0]:
## CONSTANTS

# initialize time
random.seed(time.time())

## total number of requests
num_requests = 10000

## pause after N events - for model update frequency 
n_events_per_training = 500

## time in seconds for model update frequency to catch up
model_update_frequency = 70

## sampling rate for recommendations
sampling_rate = 10

# user and context arrays to pull from randomly
namesopt = ['Alice', 'Bob', 'Cathy', 'Dave']
weatheropt = ['Sunny', 'Rainy', 'Snowy']
timeofdayopt = ['Morning', 'Afternoon', 'Evening']

# data files
examplepath = "example.json"
requestpath = "rankrequest.json"
actionfeaturespath = "actionfeatures.json"

# Replace 'personalizer_endpoint' and 'personalizer_key' with your valid endpoint values.
personalizer_endpoint = "https://westus2.cognitiveservices.azure.com"
personalizer_key = "123456789"

In [0]:
## VARIABLES

userpref = None 
rankactionsjsonobj = None 
actionfeaturesobj = None

i = 1
recommendations = 0
reward = 0
rewards = []
count = []

In [0]:
# SDK - client to Personalizer SDK, authenticated with endpoint and key
client = PersonalizerClient(personalizer_endpoint, CognitiveServicesCredentials(personalizer_key))

In [0]:
#import data

with open(examplepath) as handle:
    userpref = json.loads(handle.read())

with open(requestpath) as handle:
    rankactionsjsonobj = json.loads(handle.read())  
    
with open(actionfeaturespath) as handle:
    actionfeaturesobj = json.loads(handle.read())  


In [0]:
# pull random user and context features (read early from contextfeatures.json)
def add_random_user_and_contextfeatures(namesoption, weatheropt, timeofdayopt):   
    name = namesopt[random.randint(0,3)]
    weather = weatheropt[random.randint(0,2)]
    timeofday = timeofdayopt[random.randint(0,2)]
    return [name, weather, timeofday]

In [0]:
# create unique event id
# if you don't create one, it is created for you in the rank call
def add_event_id():
    
    # eventid is a GUID - random, unique value
    return str(uuid.uuid4())

In [0]:
def exclude_rank_actions():
    
    # replace with business logic to determine what to exclude
    # return array of action IDs
    return ['Iced mocha']

In [0]:
# determine reward
def get_reward_from_simulated_data(name, weather, timeofday, prediction):
    if(userpref[name][weather][timeofday] == str(prediction)):
        return 1 
    return 0

In [0]:
# parse JSON (read early from actionfeatures.json) to create RankableAction
def add_action_features():

    actions = []

    for action in actionfeaturesobj:
        
        # SDK - create rankable action
        rankableActon = RankableAction(id=action["id"], features=action["features"])
        
        actions.append(rankableActon) 

    return actions

In [ ]:
## LOOP - similuates traffic to your Personalizer loop

while(i <= num_requests):
      
    # reset reward to 0
    reward = 0
        
    #create unique id to associate with an event
    event_id = add_event_id()
    
    #generate a random sample, such as ['Bob', 'Snowy', 'Afternoon']
    [name, weather, timeofday] = add_random_user_and_contextfeatures(namesopt, weatheropt, timeofdayopt)
    
    #put context and environment features in JSON format SDK expects
    context_features = [{'timeofday': timeofday, 'weather': weather, 'name': name}]
    
    print(context_features)
    
    # get entire list of rankableActions 
    rankableActions = add_action_features() 
    
    # get excluded actions - based on business logic
    excluded_actions = exclude_rank_actions()

    # SDK - Create RANK request
    rank_request = RankRequest( actions=rankableActions, context_features=context_features, excluded_actions=excluded_actions, event_id=event_id) 

    # SDK - RANK - get prediction for top action
    response = client.rank(rank_request=rank_request)  
    rankedList = response.ranking
    
    ## get top ranked action's ID, such as 'Cappucino'
    prediction = response.reward_action_id
    
    print("prediction ", prediction)
  
    ## get list of ranked actions
    rankedList = response.ranking
    
    ## display ranked actions list - uncomment following 2 lines
    #for ranked in rankedList:
    #    print(ranked.id, ':',ranked.probability)
        
    #compare personalization service recommendation with the simulated data to generate a reward value
    reward = get_reward_from_simulated_data(name, weather, timeofday, prediction)
    print("reward ", reward)      
       
    # SDK - REWARD - send the reward to the service 
    client.events.reward(event_id=event_id, value=reward)
    
    # total correct recommendations 
    recommendations = recommendations + reward
    print("recommendations ", recommendations)
    
    #wait for model_update_frequency before sending more events to observe learning in the next batch
    if(i % n_events_per_training == 0):
        print("waiting for training frequency...")
        time.sleep(model_update_frequency + 5) 
               
    if(i % sampling_rate == 0):
        print("add recommendations to rewards...")
        rewards.append(recommendations)
        count.append(i)
        recommendations = 0
        
    i = i + 1
    print(i)
    
print("done with sampling")

Plot total number of correct recommendations for every batch of 10 events.

In [0]:
plt.plot(count, rewards)
plt.xlabel("Batch of rank events")
plt.ylabel("Correct recommendations per batch")
plt.show()

From the above plot, you can observe that the ranking gets better after ~2000 events and performs well over ~80% of the time. Since, the exploration is set to 20%, 20% of the time the system still tries to explore the other options. See https://docs.microsoft.com/en-us/azure/cognitive-services/personalizer/ for more documentation.